In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import netCDF4 as nc
import numpy as np
from pyproj import Proj, Transformer, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import plotly as py
from plotly.graph_objs import *
from scipy.io import netcdf
from mpl_toolkits.basemap import Basemap
import plotly.express as px
import plotly.graph_objects as go
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units
import xarray as xr

from utils import GeoTsObject

In [3]:
geo_ts = GeoTsObject()

In [4]:
dataset = geo_ts.read_netcdf(as_dataframe=False)


In [6]:
# Save data as datasets
temp = dataset['temperature'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
psurf = dataset['pressure'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
wind = dataset['wind_speed'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
qair = dataset['specific_humidity'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
rainfall = dataset['rainfall_flux'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
snowfall = dataset['snowfall_flux'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))
radiation = dataset['shortwave_down'].sel(time = slice("2000-01-01", "2015-12-31 23:00"))

In [7]:
# Conversions (see MetPy). 

precip = rainfall.assign(Precip = (rainfall.Rainf + snowfall.Snowf) * 3600)
psurf = psurf.assign(PSurf_hpa = psurf.PSurf * 0.01)

# Alternative to MetPy. Order matters here, don't change

# g_0 = 2.8365744*10**3
# g_1 = 6.028076559*10**3
# g_2 = 1.954263612*10
# g_3 = -2.737830188*10**-2
# g_4 = 1.6261698*10**-5
# g_5 = 7.0229056*10**-10
# g_6 = -1.8680009*10*-13
# g_7 = 2.7150305


# qair = qair.assign(Sat_vap_pres = pow(np.exp, (np.sum(g_0*temp.Tair + g_1*temp.Tair + g_2*temp.Tair + g_3*temp.Tair + g_4*temp.Tair +
#                                             g_5*temp.Tair + g_6*temp.Tair) + g_7* np.log(temp.Tair))))

# qair = qair.assign(Mixing_ratio = 0.622 * qair.Sat_vap_pres/ (psurf.PSurf_hpa - qair.Sat_vap_pres))

# rel_hum = qair.assign(Rel_hum = qair.Qair / ((1 - qair.Qair)* qair.Mixing_ratio))

temp = temp.assign(Tair_degc = temp.Tair - 273.15)


rel_hum = qair.assign(Rel_hum = relative_humidity_from_specific_humidity(
                        psurf.PSurf_hpa.values * units.hPa, 
                        temp.Tair_degc.values * units.degC, 
                        qair.Qair.values).to('percent').magnitude)

In [38]:
rel_hum.Rel_hum.isel(time=1).plot()

KeyboardInterrupt: 

In [5]:
# Conversions
temp = temp.update(
    {'Temp_degc': (('time', 'lat', 'lon'), temp.Tair.data - 273.15)})
precip = rainfall.update(
    {'Precip': (('time', 'lat', 'lon'), (rainfall.Rainf.data + snowfall.Snowf.data) * 3600 )})
psurf = psurf.update(
    {'Psurf_hpa': (('time', 'lat', 'lon'), psurf.PSurf.data * 0.01)})

rel_hum_values = relative_humidity_from_specific_humidity(psurf.Psurf_hpa.values * units.hPa, 
                                                            temp.Temp_degc.values * units.degC, 
                                                            qair.Qair.values).to('percent').magnitude 
rel_hum = qair.update(
    {'Rel_hum': (('time', 'lat', 'lon'), rel_hum_values * 0.01)})

KeyboardInterrupt: 

In [21]:
# Rasample to daily for efficiency
temp_m = temp.resample(time = 'M').mean()
psurf_m = psurf.resample(time = 'M').mean()
wind_m = wind.resample(time = 'M').mean()
qair_m = qair.resample(time = 'M').mean()
rainfall_m = rainfall.resample(time = 'M').sum()
snowfall_m = snowfall.resample(time = 'M').sum()
radiation_m = radiation.resample(time = 'M').mean()

In [25]:
precip_m.precip.values

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         2.41448656e-01, 7.07070708e-01, 3.76146841e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         5.44904053e-01, 1.09311736e+00, 3.42451882e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         5.06072819e-01, 2.00000000e+00, 4.69325161e+00],
        ...,
        [1.22159367e+01, 1.18451958e+01, 1.25663071e+01, ...,
         7.99158752e-01, 6.65258765e-01, 0.00000000e+00],
        [9.09593773e+00, 8.73152828e+00, 9.68501091e+00, ...,
         6.03174627e-01, 1.05319142e+00, 1.86123347e+00],
        [9.04057884e+00, 9.40672016e+00, 9.64348030e+00, ...,
         6.67420805e-01, 9.09091055e-01, 1.13483155e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         7.14395218e+01, 8.83932190e+01, 9.35263824e+01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         6.46339111e+01, 8.01007004e+01, 9.82021866e+01],
        [0.00000000e+00, 